In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import theano
import theano.tensor as T
import lasagne
import time

In [ ]:
'''x_train
y_train
x_test
y_test
'''
#load the data 

In [ ]:
#Define a simple feed forward neural network
from lasagne.nonlinearities import leaky_rectify, softmax, tanh, elu
from lasagne.layers import InputLayer, DenseLayer, Conv2DLayer, batch_norm, DropoutLayer, MaxPool2DLayer
from lasagne.layers import batch_norm
import math

#defined the model
num_class = 10
num_features = 28*28
init=math.sqrt(2/(1+0.01**2))
l_in=InputLayer( shape=(None,1,28,28) )
#note that we use a 4D input since we need to retain the spatial arrangement of the pixels when working with convolutions.

l_conv = (Conv2DLayer(l_in,num_filters=128,filter_size=3,nonlinearity=leaky_rectify, 
                     W=lasagne.init.GlorotUniform(init)))
ld1= lasagne.layers.dropout(l_conv, p=0)
l_conv2 = batch_norm(lasagne.layers.Conv2DLayer( ld1, num_filters=64, filter_size=(3, 3),
            nonlinearity=leaky_rectify, W=lasagne.init.GlorotUniform(init)))
ld2= lasagne.layers.dropout(l_conv2, p=0)

l_conv3 = batch_norm(lasagne.layers.Conv2DLayer(ld2, num_filters=64, filter_size=(3, 3),
            nonlinearity=leaky_rectify, W=lasagne.init.GlorotUniform(init)))
ld3= lasagne.layers.dropout(l_conv3, p=0)
l_conv4 = batch_norm(lasagne.layers.Conv2DLayer(ld3, num_filters=32, filter_size=(3, 3),
            nonlinearity=leaky_rectify, W=lasagne.init.GlorotUniform(init)))
ld4= lasagne.layers.dropout(l_conv4, p=0)
    
l_hid = batch_norm(DenseLayer( ld4, num_units=1000, nonlinearity=leaky_rectify,W=lasagne.init.GlorotUniform(init))) #remember to connect the new conv-layer here
l_hid2 = batch_norm(DenseLayer( lasagne.layers.dropout(l_hid, p=0), 
                                num_units=600, nonlinearity=leaky_rectify,W=lasagne.init.GlorotUniform(init))) #remember to connect the new conv-layer here
l_hid3 = batch_norm(DenseLayer( lasagne.layers.dropout(l_hid2,  p=0), 
                                num_units=300, nonlinearity=leaky_rectify,W=lasagne.init.GlorotUniform(init))) #remember to connect the new conv-layer here
l_hid4 = batch_norm(DenseLayer( lasagne.layers.dropout(l_hid3,  p=0), 
                                num_units=150, nonlinearity=leaky_rectify,W=lasagne.init.GlorotUniform(init))) #remember to connect the new conv-layer here
l_out = DenseLayer( lasagne.layers.dropout(l_hid4,  p=0), num_units=num_class, nonlinearity=softmax)

In [ ]:
from lasagne.regularization import regularize_layer_params_weighted, l2, l1
sym_x = T.tensor4('sym_x') # a symbolic variable taking on the value of a input batch.
sym_t = T.ivector('sym_t') # a symbolic variable taking on the value of the target batch.

train_out = lasagne.layers.get_output(l_out, sym_x, deterministic=False)
eval_out = lasagne.layers.get_output(l_out, sym_x, deterministic=True)
all_params = lasagne.layers.get_all_params(l_out, trainable=True)
cost = T.nnet.categorical_crossentropy(train_out+1e-8, sym_t).mean()

all_grads = T.grad(cost, all_params)


updates = lasagne.updates.adadelta(all_grads, all_params, learning_rate=0.01)

f_eval = theano.function([sym_x],
                     eval_out, on_unused_input='warn')

f_train = theano.function([sym_x, sym_t],
                          [cost],
                          updates=updates, on_unused_input='warn')

In [ ]:
import sys
num_epochs=2
num_classes=10
train_acc, train_loss = [], []
valid_acc, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
loss = []
batch_size = 100
num_epochs = 2
learning_rate = 0.1
num_samples_train = x_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_batches_train=500
x_train=x_train.reshape((-1,1,28,28))
for epoch in range(num_epochs):
        cur_loss = 0
        for i in range(num_batches_train):
            if i%20==0:
                print i/20,
            idx = range(i*batch_size, (i+1)*batch_size)
            x_batch = x_train[idx]
            target_batch = y_train[idx]    
            batch_loss = f_train(x_batch,target_batch) #this will do the complete backprob pass
            cur_loss += batch_loss[0]
        loss += [cur_loss/batch_size]

        for i in range(num_batches_train):
            idx = range(i*batch_size, (i+1)*batch_size)
            x_batch = x_train[idx]
            targets_batch = y_train[idx]
            net_out = f_eval(x_batch)   
            preds = np.argmax(net_out, axis=-1) 


        train_acc += [train_acc_cur]
        print "Epoch %i : Train Loss %e , Train acc %f" \
        % (epoch+1, loss[-1], train_acc_cur)



epoch = np.arange(len(train_acc))
plt.figure()
plt.plot(epoch,train_acc,'r')
plt.legend(['Train Acc'])
plt.xlabel('Epochs'), plt.ylabel('Acc'), plt.ylim([0.75,1.03])
